In [1]:
import csv
import math
import pandas as pd
from pathlib import Path
from typing import Dict
from census import Census
from pathlib import Path
from api_keys import census_key

In [2]:
county_dfs: Dict[str, pd.DataFrame] = {}

for path in Path("Budget").glob("Budget_by_County_*.csv"):
    name_parts = path.stem.split("_")
    year = name_parts[-1]
    if year == "Merged":
        continue
    df = pd.read_csv(path)
    df["Year"] = int(year)
    for row in range(len(df.index)):
        df.iloc[row, 0] = df.iloc[row, 0].strip(" ").title()
    county_dfs[year] = df

df = pd.DataFrame(columns=county_dfs["2010"].columns)

for year, year_df in county_dfs.items():
    for col in [col for col in year_df.columns if col not in ["State","County/ Congressional District"]]:
        year_df[col] = pd.to_numeric(year_df[col], errors="coerce")
        year_df = year_df.fillna(0)
        year_df[col] = year_df[col].astype("int")
    df = df.merge(year_df, how="outer")
merged_county_df = df

merged_county_df.to_csv(f"Budget/Budget_by_County_Merged.csv", index=False)

In [3]:
state_dfs: Dict[str, pd.DataFrame] = {}

for path in Path("Budget").glob("State_Total_*.csv"):
    name_parts = path.stem.split("_")
    year = name_parts[-1]
    if year == "Merged":
        continue
    df = pd.read_csv(path)
    df["Year"] = int(year)
    for row in range(len(df.index)):
        df.iloc[row, 0] = df.iloc[row, 0].strip(" ").title()
    state_dfs[year] = df

df = pd.DataFrame(columns=state_dfs["2010"].columns)

for year, year_df in state_dfs.items():
    for col in [col for col in year_df.columns if col != "State"]:
        year_df[col] = pd.to_numeric(year_df[col], errors="coerce")
        year_df = year_df.fillna(0)
        year_df[col] = year_df[col].astype("int")
    df = df.merge(year_df, how="outer")

df.to_csv(f"Budget/State_Total_Merged.csv", index=False)

In [4]:
# Find the veteran population from the Census API
year_dfs = []

for year in range(2010, 2016):
    year_data = Census(census_key, year=year).acs5.get(
        ( "NAME", "B21001_002E" ),
        { 'for': 'county:*', 'in': 'state:*' }
    )

    year_df = pd.DataFrame(year_data).rename(
        columns = {
            'county': 'County/ Congressional District',
            'state': 'State',
            'B21001_002E': 'Veteran Population'
        }
    )[[
        "State",
        "County/ Congressional District",
        "Veteran Population"
    ]]

    year_df["Year"] = year
    year_df["County/ Congressional District"] = (year_df["State"] + year_df["County/ Congressional District"]).astype("int")
    year_df["State"] = year_df["State"].astype("int")
    year_df["Veteran Population"] = year_df["Veteran Population"].astype("int")
    
    year_dfs.append(year_df)

census_df = pd.DataFrame(columns=year_dfs[0].columns)

for year_df in year_dfs:
    census_df = census_df.merge(year_df, how='outer')

fips_dict = {}
with Path("county_fips_master.csv").open(encoding='cp1252') as f:
    header_fields = f.readline().split(",")
    reader = csv.DictReader(f, fieldnames=header_fields)
    for row in reader:
        county_fips = int(row["fips"])
        fips_dict[county_fips] = row["county_name"].replace(" City and Borough", "").replace(" Census Area", "").replace(" Borough", "").replace(" County", "").replace(" Municipality", "").replace(bytes.fromhex("D0").decode("cp1252"),"n").replace("á","a").replace("í","i").replace("ó","o").replace("ü","u").replace("ñ","n").replace("St.","Saint").upper()
        state_fips = math.floor(county_fips / 1000)
        fips_dict[state_fips] = row["state_name"].title()

census_df = census_df.replace({ "State": fips_dict, "County/ Congressional District": fips_dict })
census_df.to_csv(f"Population/census_data.csv", index=False)